In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load in 

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the "../input/" directory.
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('../kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # Any results you write to the current directory are saved as output.

이 튜토리얼에서 우리는 베이직한 자연어처리(NLP) 기법 몇 개를 다룰 것입니다. 다룰 목록은 다음과 같습니다:
* 유용한 일부 NLP 라이브러리 & 데이터셋 읽기
* 각 작가들이 각 단어를 사용한 빈도수 찾기
* 어느 작가가 문장을 썼는지 추측하는 데에 단어 빈도수 사용하기

준비되었나요? 시작해봅시다! :D

# 개요
해당 튜토리얼에서는 정규화된 유니그램 빈도를 기준으로 어느 작가가 텍스트 문자열을 작성했는지 추측합니다. 이는 각 작가가 훈련 데이터에서 모든 단어를 얼마나 자주 사용하는지를 세고 그들이 쓴 총 단어 갯수로 나누는 멋진 방법입니다. 그런 다음, 테스트 문장에서 한 작가가 다른 작가들보다 더 많은 단어를 사용하는 것을 본다면 우리는 이 작가가 그 작가임을 추측할 수 있습니다.
다음을 우리의 트레이닝 말뭉치라고 가정해봅시다:

* 작가 1: "A very spooky thing happened. The thing was so spooky I screamed."
* 작가 2: "I ate a tasty candy apple. It was delicious"

그리고 다음은 어느 작가가 썼는지 알아내고 싶은 테스트 문장입니다:

* 작가 ??? : "What a spooky thing!"

그냥 봐서는, 이 문장은 작가 1이 쓴 것 같습니다. 작가 1은 'spooky'와 'thing'을 둘 다 많이 쓰는 반면, 작가 2는 그렇지 않습니다(적어도 우리의 훈련 데이터에 따르면). 테스트 문장에서 'spooky'와 'thing'을 둘 다 볼 수 있기 때문에, 이는 작가 2보다는 작가 1이 쓴 것이라고 볼 수 있습니다 -- 비록 테스트 문장에는 'a'라는 단어가 있고 작가 2도 이를 사용했지만 말입니다. 

튜토리얼의 나머지 부분에서는 이 직관을 코드로 변환할 수 있는지 알아볼 것입니다.

# 유용한 NLP 라이브러리 & 데이터셋

이 튜토리얼에서는 'NLTK' 라고 하는 Natural Language Toolkit을 사용하겠습니다. 이는 언어 데이터를 분석하기 위한 오픈 소스 파이썬 라이브러리입니다. NLTK의 좋은 점은 많은 일반적인 NLP 작업을 단계별로 진행하는 유용한 책이 있다는 것입니다. 더 좋은 점은 여기에서 책을 무료로 얻을 수 있다는 것입니다.

In [2]:
# read in some helpful libraries
import nltk # the natural language toolkit, open-source NLP
import pandas as pd # dataframes
import zipfile

### Read in the data

# read our data into a dataframe
df = pd.DataFrame()
#pd.read_csv() - df = pd.read_csv('../input/spooky-author-identification/train.zip', compression='zip', header=0, sep=',', quotechar='"')
Dataset = 'train'

# Will unzip the files so that you can see them..
with zipfile.ZipFile("../input/spooky-author-identification/"+Dataset+".zip","r") as z:
    z.extractall(".")

texts = pd.read_csv(Dataset + '.csv')

# look at the first few rows
texts.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [3]:
#print(os.path.join(dirname, filename))

In [4]:
#import os
#print(os.listdir('../input/spooky-author-identification/train.zip'))

# 각 작가가 각 단어를 사용한 빈도 수 알아보기

각 작가가 각 단어를 얼마나 자주 사용하는지 알아보십시오. 많은 NLP 응용 프로그램은 특정 단어가 얼마나 자주 사용되는지 계산하는 데 의존합니다. (이 용어에 대한 멋진 용어는 '단어 빈도' 입니다.) 데이터셋에서 각 저자의 단어 빈도를 살펴보겠습니다. NLTK에는 이를 위해 사용할 수 있는 멋진 내장함수와 데이터 구조가 많이 있습니다.

In [5]:
### Split data

# split the data by author
byAuthor = texts.groupby('author')

### Tokenize (split into individual words) our text

# word frequency by author
wordFreqByAuthor = nltk.probability.ConditionalFreqDist()

# for each author...
for name, group in byAuthor:
    # get all of the sentences they wrote and collapse them into a
    # single long string
    sentences = group['text'].str.cat(sep = ' ')
    
    # convert everything to lower case (so 'The' and 'the' get counted as
    # the same word rather than two different words)
    sentences = sentences.lower()
    
    # split the text into individual tokens
    tokens = nltk.tokenize.word_tokenize(sentences)
    
    # calculate the frequency of each token
    frequency = nltk.FreqDist(tokens)
    
    # add the frequencies for each author to our dictionary
    wordFreqByAuthor[name] = (frequency)
    
# now we have an dictionary where each entry is the frequency distribution
# of words for a specific author

이제 우리는 각 작가가 특정 단어를 어느 빈도로 사용하는지 볼 수 있습니다. 이것이 할로윈 대회용이니까 'blood', 'scream', 'fear' 같은 건 어떨까요?

In [6]:
# see how often each author says 'blood'
for i in wordFreqByAuthor.keys():
    print('blood: ' + i)
    print(wordFreqByAuthor[i].freq('blood'))

# print a blank line
print()

# see how often each author says 'scream'
for i in wordFreqByAuthor.keys():
    print('scream: ' + i)
    print(wordFreqByAuthor[i].freq('scream'))
    
# print a blank line
print()

# see how often each author says 'fear'
for i in wordFreqByAuthor.keys():
    print('fear: ' + i)
    print(wordFreqByAuthor[i].freq('fear'))

blood: EAP
0.00014646397201676582
blood: HPL
0.00022992337803427008
blood: MWS
0.00022773011333545174

scream: EAP
1.7231055531384214e-05
scream: HPL
9.196935121370803e-05
scream: MWS
2.6480245736680435e-05

fear: EAP
0.00010338633318830528
fear: HPL
0.0005748084450856752
fear: MWS
0.0006196377502383222


# 어느 작가가 문장을 썼는지 추측하는 데에 단어 빈도수 사용하기

일반적인 아이디어는 다른 사람들은 다른 단어들을 더 자주 또는 덜 사용하는 경향이 있다는 것입니다. (나의 경우에는 특히 'gestalt'를 좋아하셨던 교수님이 계셨습니다.) 누가 무엇을 말했는지 확실하지 않지만 그 안에 한 사람이 많이 사용하는 단어가 많을 경우, 그 단어들은 어떤 한 사람이 썼을 것이라고 추측 가능합니다.

이 일반적인 원칙을 사용하여 누가 'It was a dark and stormy night.'이라는 문장을 쓸 가능성이 더 높을지 추측해봅시다.

In [7]:
# One way to guess authorship is to use the joint probability that each
# author used each word in a given sentence

# first, let's start with a test sentence
testSentence = "It was a dark and stormy night."

# and then lowercase & tokenize our test sentence
preProcessedTestSentence = nltk.tokenize.word_tokenize(testSentence.lower())

# create an empty dataframe to put our output in
testProbabilities = pd.DataFrame(columns = ['author', 'word','probability'])

# For each author...
for i in wordFreqByAuthor.keys():
    # for each word in our test sentence..
    for j in preProcessedTestSentence:
        # find out how frequentyly the author used that word
        wordFreq = wordFreqByAuthor[i].freq(j)
        # and add a very small amount to every prob. so none of them are 0
        smoothedWordFreq = wordFreq + 0.000001
        # add the author, word and smoothed freq. to our dataframe
        output = pd.DataFrame([[i,j,smoothedWordFreq]], columns = ['author', 'word','probability'])
        testProbabilities = testProbabilities.append(output, ignore_index = True)

# empty dataframe for the probability that each author wrote the snetence
testProbabilitiesByAuthor = pd.DataFrame(columns = ['author','jointProbability'])

# now let's group the dataframe with our frequency by author
for i in wordFreqByAuthor.keys():
    # get the joint probability that each author wrote each word
    oneAuthor = testProbabilities.query('author == "' + i + '"')
    jointProbability = oneAuthor.product(numeric_only = True)[0]
    
    # and add that to our dataframe
    output = pd.DataFrame([[i, jointProbability]], columns = ['author','jointProbability'])
    testProbabilitiesByAuthor = testProbabilitiesByAuthor.append(output, ignore_index=True)
    
# and our winner is...
testProbabilitiesByAuthor.loc[testProbabilitiesByAuthor['jointProbability'].idxmax(),'author']

'HPL'

우리가 훈련 데이터에서 본 것을 바탕으로, 세 명의 작가들 중 H.P.Lovecraft가 'It was a dark and stormy night" 라는 문장을 썼을 가능성이 높은 것으로 나타납니다.

In [8]:
testProbabilitiesByAuthor.to_csv("testProbabilityByAuthor.csv", index=False)